In [1]:
#!pip3 install "z3-solver"
#https://wiki.mma.club.uec.ac.jp/CTF/Toolkit/z3py#A.2BUjZ9BDBuii1bmg.28.2BMNMwwzDIMNkwrzDIMOs.29

In [2]:
#import 
import numpy as np
import pandas as pd
import random
from z3 import *

In [4]:
def calc_marginals(df):                   
    return np.array([                      
        sum(df['Y']),                     
        np.dot(df['Y'], df['LI']),      
        np.dot(df['Y'], df['SEX']),      
        np.dot(df['Y'], df['AOP']),      
    ])

In [6]:
def n_solutions_all(df):
    y_dic = {}
    for t1 in range(sum(df['LI'])):
        y_dic[t1] = []
        for _ in range(2**df.shape[0]):
            
            t_list = calc_marginals(df)
            s = Solver()
            y = [Int("y[%s]" % i) for i in range(df.shape[0])]
    
            for i in range(df.shape[0]):
                s.add(And(0 <= y[i], y[i] <= 1))
        
            s.add(And(sum(y)==int(t_list[0])))
            s.add(And(sum([df.iloc[i, 1]*y[i] for i in range(df.shape[0])])==t1))
    
            for j in range(2, df.shape[1]):
                s.add(And(sum([df.iloc[i, j]*y[i] for i in range(df.shape[0])])==int(t_list[j])))
        
            if s.check() == sat:
                m = s.model()
                #print(m)
                new_y = [m.evaluate(y[i]) for i in range(df.shape[0])]
                if not new_y in y_dic[t1]:
                    y_dic[t1].append(new_y)
    return y_dic

In [9]:
def n_solutions_each(df, t1):
    y_list= []
    for _ in range(2**df.shape[0]):

        t_list = calc_marginals(df)
        s = Solver()
        y = [Int("y[%s]" % i) for i in range(df.shape[0])]

        for i in range(df.shape[0]):
            s.add(And(0 <= y[i], y[i] <= 1))

        s.add(And(sum(y)==int(t_list[0])))
        s.add(And(sum([df.iloc[i, 1]*y[i] for i in range(df.shape[0])])==t1))

        for j in range(2, df.shape[1]):
            s.add(And(sum([df.iloc[i, j]*y[i] for i in range(df.shape[0])])==int(t_list[j])))

        if s.check() == sat:
            m = s.model()
            new_y = [m.evaluate(y[i]) for i in range(df.shape[0])]
            if not new_y in y_list:
                y_list.append(new_y)
    return y_list

In [ ]:
def n_solutions_each(df, t1):
    loop = 10
    result = timeit.timeit('n_solutions_all(df)', globals=globals(), number=loop)
    return result / loop